# 영상 받아서 이미지 한장씩 저장하기

In [1]:
import cv2
import os

filepath = './test111.mp4'
video = cv2.cv2.VideoCapture(filepath)

if not video.isOpened():
    print("Could not Open :",filepath)
    exit(0)
    
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 2004
width : 1920
height : 1080
fps : 30.0


In [2]:
new_path = './new_image/' # 이미지가 담길 폴더
import cv2
vidcap = cv2.VideoCapture(filepath)  # 영상이 있는 경로 
count = 0 

while(vidcap.isOpened()): 
    ret, image = vidcap.read() # 이미지 사이즈 960x540으로 변경 
    if ret:
        # image = cv2.resize(image, (960, 540)) # 30프레임당 하나씩 이미지 추출 
        if(int(vidcap.get(1)) % 30 == 0): 
            print('Saved frame number : ' + str(int(vidcap.get(1)))) # 추출된 이미지가 저장되는 경로 
            cv2.imwrite(new_path+"frame%d.png" % count, image) #
            print('Saved frame%d.jpg' % count) 
            count += 1
    else:
        break
        
vidcap.release()

Saved frame number : 30
Saved frame0.jpg
Saved frame number : 60
Saved frame1.jpg
Saved frame number : 90
Saved frame2.jpg
Saved frame number : 120
Saved frame3.jpg
Saved frame number : 150
Saved frame4.jpg
Saved frame number : 180
Saved frame5.jpg
Saved frame number : 210
Saved frame6.jpg
Saved frame number : 240
Saved frame7.jpg
Saved frame number : 270
Saved frame8.jpg
Saved frame number : 300
Saved frame9.jpg
Saved frame number : 330
Saved frame10.jpg
Saved frame number : 360
Saved frame11.jpg
Saved frame number : 390
Saved frame12.jpg
Saved frame number : 420
Saved frame13.jpg
Saved frame number : 450
Saved frame14.jpg
Saved frame number : 480
Saved frame15.jpg
Saved frame number : 510
Saved frame16.jpg
Saved frame number : 540
Saved frame17.jpg
Saved frame number : 570
Saved frame18.jpg
Saved frame number : 600
Saved frame19.jpg
Saved frame number : 630
Saved frame20.jpg
Saved frame number : 660
Saved frame21.jpg
Saved frame number : 690
Saved frame22.jpg
Saved frame number : 720

# 연속된 이미지 불러와서 검출 -> 분류 -> 시각화

- 연속된 이미지 파일명 : 20200602_1.jpg (이런식으로 파일명을 변경함) 
- 여기서 검출된 신호등 20200602_1_1, 20200602_1_2 ... 이런식으로

In [3]:
import cv2
import matplotlib.pyplot as plt

## green yellow red 모델들

# 파일명 시작 숫자 설정
seq = 1 # 채번 변수 통일 20220308 by yjlim

# 추출 이미지 저장 위치 설정
# 이미지 한장으로 테스트 하는 거라서 디렉토리 분류 할 필요 없이 한 폴더에 다 담기
path = './vi_data2/'

first_detect = []

##### 1. green #####
def green(img, name, X, Y, W, H):
    global seq, path_g, path
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    name = name.split('.')
    
    # ROI 지정
    roi = B[Y:Y+H, X:X+W]
    roi2 = roi.copy()    

    ## 초록색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (43, 40, 120)
    up = (95, 255, 255)

    # 범위내의 픽셀들은 흰색, 나머지 검은색
    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    
    # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion2 = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)    
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)
    

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))
            
    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - 2 * length - 2 * interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,\
                        X + a[0] - 2 * length - 2 * interval_x:X + a[0] + length + interval_x]
                print(img + '///G' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y, a[1] + a[3] + interval_y, X + a[0] - 2 * length - 2 * interval_x, X + a[0] + length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 2. yellow #####
def yellow(img, name, X, Y, W, H):
    global seq, path_y, path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H, X:X+W]
    roi2 = roi.copy()

    ## 노랑색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (12, 10, 110)
    up = (31, 255, 255)
    # low = (20, 20, 100)
    # up = (30, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    
    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - length - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - length - interval_x:X + a[0] + 2 * length + interval_x]
                
                print(img + '///Y' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y,X + a[0] - length - interval_x, X + a[0] + 2 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 3. red #####
def red(img, name, X, Y, W, H):
    global seq, path_r,path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H,X:X+W]
    roi2 = roi.copy()

    ## 빨강색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (0, 30, 50)
    up = (10, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    # 색 영역 두번째 추가
    low3 = (150, 40, 50)
    up3 = (180, 255, 255)
    
    roi3_mask = cv2.inRange(roi2_hsv, low3, up3)
    roi3_result = cv2.bitwise_and(roi2, roi2, mask = roi3_mask)
    
    # 색 영역 1 + 색 영역 2  
    result_red = roi2_result + roi3_result

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(result_red, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(result_red, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3)
            interval_y = int(a[3] / 4)
            
            if a[1] - interval_y > 0 and X + a[0] - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - interval_x:X + a[0] + 3 * length + interval_x]
                print(img + '///R' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y, X + a[0] - interval_x, X + a[0] + 3 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] +'_'+ str(seq) + '.jpg', real3)
                seq = seq + 1

In [4]:
import sys
import glob
import cv2

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
# 연속된 이미지가 담긴 폴더
#img_files = glob.glob('./video/*.jpg')
img_files = glob.glob('.\\new_image\\*.png')
img_files.sort() # 순서대로 정렬

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
    
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
    if cv2.waitKey(1000) == 27:     
        break

    index += 1      
    
    # ROI 지정, x,y,w,h ,N 설정 완료
    x1 = 300
    y1 = 0
    w1 = 1400
    h1 = 640
    
    green(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    yellow(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    red(filepath, name, X = x1, Y = y1, W = w1, H = h1)


.\new_image\frame0.png///G1.jpg
.\new_image\frame0.png///G2.jpg
.\new_image\frame0.png///G3.jpg
.\new_image\frame0.png///G4.jpg
.\new_image\frame0.png///G5.jpg
.\new_image\frame0.png///G6.jpg
.\new_image\frame0.png///Y7.jpg
.\new_image\frame0.png///Y8.jpg
.\new_image\frame0.png///Y9.jpg
.\new_image\frame0.png///Y10.jpg
.\new_image\frame0.png///Y11.jpg
.\new_image\frame0.png///R12.jpg
.\new_image\frame0.png///R13.jpg
.\new_image\frame0.png///R14.jpg
.\new_image\frame0.png///R15.jpg
.\new_image\frame0.png///R16.jpg
.\new_image\frame0.png///R17.jpg
.\new_image\frame0.png///R18.jpg
.\new_image\frame0.png///R19.jpg
.\new_image\frame0.png///R20.jpg
.\new_image\frame0.png///R21.jpg
.\new_image\frame1.png///G22.jpg
.\new_image\frame1.png///G23.jpg
.\new_image\frame1.png///G24.jpg
.\new_image\frame1.png///G25.jpg
.\new_image\frame1.png///Y26.jpg
.\new_image\frame1.png///Y27.jpg
.\new_image\frame1.png///Y28.jpg
.\new_image\frame1.png///Y29.jpg
.\new_image\frame1.png///Y30.jpg
.\new_image\frame1.

.\new_image\frame18.png///G256.jpg
.\new_image\frame18.png///G257.jpg
.\new_image\frame18.png///G258.jpg
.\new_image\frame18.png///Y259.jpg
.\new_image\frame18.png///Y260.jpg
.\new_image\frame18.png///Y261.jpg
.\new_image\frame18.png///Y262.jpg
.\new_image\frame18.png///R263.jpg
.\new_image\frame19.png///Y264.jpg
.\new_image\frame19.png///R265.jpg
.\new_image\frame19.png///R266.jpg
.\new_image\frame2.png///G267.jpg
.\new_image\frame2.png///G268.jpg
.\new_image\frame2.png///G269.jpg
.\new_image\frame2.png///Y270.jpg
.\new_image\frame2.png///Y271.jpg
.\new_image\frame2.png///Y272.jpg
.\new_image\frame2.png///Y273.jpg
.\new_image\frame2.png///Y274.jpg
.\new_image\frame2.png///Y275.jpg
.\new_image\frame2.png///Y276.jpg
.\new_image\frame2.png///Y277.jpg
.\new_image\frame2.png///Y278.jpg
.\new_image\frame2.png///Y279.jpg
.\new_image\frame2.png///Y280.jpg
.\new_image\frame2.png///R281.jpg
.\new_image\frame2.png///R282.jpg
.\new_image\frame2.png///R283.jpg
.\new_image\frame2.png///R284.jpg
.\n

.\new_image\frame28.png///Y511.jpg
.\new_image\frame28.png///Y512.jpg
.\new_image\frame28.png///Y513.jpg
.\new_image\frame28.png///Y514.jpg
.\new_image\frame28.png///Y515.jpg
.\new_image\frame28.png///Y516.jpg
.\new_image\frame28.png///Y517.jpg
.\new_image\frame28.png///Y518.jpg
.\new_image\frame28.png///Y519.jpg
.\new_image\frame28.png///Y520.jpg
.\new_image\frame28.png///Y521.jpg
.\new_image\frame28.png///Y522.jpg
.\new_image\frame28.png///Y523.jpg
.\new_image\frame28.png///Y524.jpg
.\new_image\frame28.png///Y525.jpg
.\new_image\frame28.png///Y526.jpg
.\new_image\frame28.png///Y527.jpg
.\new_image\frame28.png///Y528.jpg
.\new_image\frame28.png///R529.jpg
.\new_image\frame28.png///R530.jpg
.\new_image\frame28.png///R531.jpg
.\new_image\frame28.png///R532.jpg
.\new_image\frame28.png///R533.jpg
.\new_image\frame28.png///R534.jpg
.\new_image\frame28.png///R535.jpg
.\new_image\frame28.png///R536.jpg
.\new_image\frame28.png///R537.jpg
.\new_image\frame28.png///R538.jpg
.\new_image\frame28.

.\new_image\frame34.png///Y757.jpg
.\new_image\frame34.png///Y758.jpg
.\new_image\frame34.png///Y759.jpg
.\new_image\frame34.png///Y760.jpg
.\new_image\frame34.png///Y761.jpg
.\new_image\frame34.png///Y762.jpg
.\new_image\frame34.png///Y763.jpg
.\new_image\frame34.png///Y764.jpg
.\new_image\frame34.png///Y765.jpg
.\new_image\frame34.png///Y766.jpg
.\new_image\frame34.png///Y767.jpg
.\new_image\frame34.png///Y768.jpg
.\new_image\frame34.png///Y769.jpg
.\new_image\frame34.png///Y770.jpg
.\new_image\frame34.png///Y771.jpg
.\new_image\frame34.png///Y772.jpg
.\new_image\frame34.png///Y773.jpg
.\new_image\frame34.png///Y774.jpg
.\new_image\frame34.png///Y775.jpg
.\new_image\frame34.png///Y776.jpg
.\new_image\frame34.png///Y777.jpg
.\new_image\frame34.png///Y778.jpg
.\new_image\frame34.png///Y779.jpg
.\new_image\frame34.png///Y780.jpg
.\new_image\frame34.png///Y781.jpg
.\new_image\frame34.png///Y782.jpg
.\new_image\frame34.png///R783.jpg
.\new_image\frame34.png///R784.jpg
.\new_image\frame34.

.\new_image\frame4.png///Y1041.jpg
.\new_image\frame4.png///Y1042.jpg
.\new_image\frame4.png///Y1043.jpg
.\new_image\frame4.png///Y1044.jpg
.\new_image\frame4.png///Y1045.jpg
.\new_image\frame4.png///Y1046.jpg
.\new_image\frame4.png///Y1047.jpg
.\new_image\frame4.png///Y1048.jpg
.\new_image\frame4.png///Y1049.jpg
.\new_image\frame4.png///Y1050.jpg
.\new_image\frame4.png///Y1051.jpg
.\new_image\frame4.png///Y1052.jpg
.\new_image\frame4.png///Y1053.jpg
.\new_image\frame4.png///Y1054.jpg
.\new_image\frame4.png///Y1055.jpg
.\new_image\frame4.png///Y1056.jpg
.\new_image\frame4.png///Y1057.jpg
.\new_image\frame4.png///Y1058.jpg
.\new_image\frame4.png///R1059.jpg
.\new_image\frame4.png///R1060.jpg
.\new_image\frame4.png///R1061.jpg
.\new_image\frame4.png///R1062.jpg
.\new_image\frame4.png///R1063.jpg
.\new_image\frame40.png///G1064.jpg
.\new_image\frame40.png///G1065.jpg
.\new_image\frame40.png///G1066.jpg
.\new_image\frame40.png///G1067.jpg
.\new_image\frame40.png///G1068.jpg
.\new_image\fra

...

.\new_image\frame53.png///R2611.jpg
.\new_image\frame53.png///R2612.jpg
.\new_image\frame53.png///R2613.jpg
.\new_image\frame53.png///R2614.jpg
.\new_image\frame53.png///R2615.jpg
.\new_image\frame53.png///R2616.jpg
.\new_image\frame53.png///R2617.jpg
.\new_image\frame53.png///R2618.jpg
.\new_image\frame53.png///R2619.jpg
.\new_image\frame53.png///R2620.jpg
.\new_image\frame53.png///R2621.jpg
.\new_image\frame53.png///R2622.jpg
.\new_image\frame53.png///R2623.jpg
.\new_image\frame53.png///R2624.jpg
.\new_image\frame53.png///R2625.jpg
.\new_image\frame53.png///R2626.jpg
.\new_image\frame54.png///G2627.jpg
.\new_image\frame54.png///G2628.jpg
.\new_image\frame54.png///G2629.jpg
.\new_image\frame54.png///G2630.jpg
.\new_image\frame54.png///G2631.jpg
.\new_image\frame54.png///G2632.jpg
.\new_image\frame54.png///G2633.jpg
.\new_image\frame54.png///G2634.jpg
.\new_image\frame54.png///G2635.jpg
.\new_image\frame54.png///G2636.jpg
.\new_image\frame54.png///G2637.jpg
.\new_image\frame54.png///G2

.\new_image\frame56.png///R2935.jpg
.\new_image\frame57.png///G2936.jpg
.\new_image\frame57.png///G2937.jpg
.\new_image\frame57.png///G2938.jpg
.\new_image\frame57.png///G2939.jpg
.\new_image\frame57.png///G2940.jpg
.\new_image\frame57.png///G2941.jpg
.\new_image\frame57.png///G2942.jpg
.\new_image\frame57.png///G2943.jpg
.\new_image\frame57.png///G2944.jpg
.\new_image\frame57.png///G2945.jpg
.\new_image\frame57.png///G2946.jpg
.\new_image\frame57.png///G2947.jpg
.\new_image\frame57.png///G2948.jpg
.\new_image\frame57.png///Y2949.jpg
.\new_image\frame57.png///Y2950.jpg
.\new_image\frame57.png///Y2951.jpg
.\new_image\frame57.png///Y2952.jpg
.\new_image\frame57.png///Y2953.jpg
.\new_image\frame57.png///Y2954.jpg
.\new_image\frame57.png///Y2955.jpg
.\new_image\frame57.png///Y2956.jpg
.\new_image\frame57.png///Y2957.jpg
.\new_image\frame57.png///Y2958.jpg
.\new_image\frame57.png///Y2959.jpg
.\new_image\frame57.png///Y2960.jpg
.\new_image\frame57.png///Y2961.jpg
.\new_image\frame57.png///Y2

.\new_image\frame59.png///R3264.jpg
.\new_image\frame59.png///R3265.jpg
.\new_image\frame59.png///R3266.jpg
.\new_image\frame59.png///R3267.jpg
.\new_image\frame59.png///R3268.jpg
.\new_image\frame59.png///R3269.jpg
.\new_image\frame59.png///R3270.jpg
.\new_image\frame59.png///R3271.jpg
.\new_image\frame59.png///R3272.jpg
.\new_image\frame59.png///R3273.jpg
.\new_image\frame59.png///R3274.jpg
.\new_image\frame59.png///R3275.jpg
.\new_image\frame59.png///R3276.jpg
.\new_image\frame59.png///R3277.jpg
.\new_image\frame59.png///R3278.jpg
.\new_image\frame59.png///R3279.jpg
.\new_image\frame59.png///R3280.jpg
.\new_image\frame59.png///R3281.jpg
.\new_image\frame59.png///R3282.jpg
.\new_image\frame59.png///R3283.jpg
.\new_image\frame6.png///Y3284.jpg
.\new_image\frame6.png///Y3285.jpg
.\new_image\frame6.png///R3286.jpg
.\new_image\frame6.png///R3287.jpg
.\new_image\frame6.png///R3288.jpg
.\new_image\frame6.png///R3289.jpg
.\new_image\frame6.png///R3290.jpg
.\new_image\frame6.png///R3291.jpg


.\new_image\frame62.png///G3538.jpg
.\new_image\frame62.png///G3539.jpg
.\new_image\frame62.png///G3540.jpg
.\new_image\frame62.png///G3541.jpg
.\new_image\frame62.png///G3542.jpg
.\new_image\frame62.png///G3543.jpg
.\new_image\frame62.png///G3544.jpg
.\new_image\frame62.png///G3545.jpg
.\new_image\frame62.png///G3546.jpg
.\new_image\frame62.png///G3547.jpg
.\new_image\frame62.png///G3548.jpg
.\new_image\frame62.png///G3549.jpg
.\new_image\frame62.png///G3550.jpg
.\new_image\frame62.png///G3551.jpg
.\new_image\frame62.png///G3552.jpg
.\new_image\frame62.png///G3553.jpg
.\new_image\frame62.png///G3554.jpg
.\new_image\frame62.png///G3555.jpg
.\new_image\frame62.png///Y3556.jpg
.\new_image\frame62.png///Y3557.jpg
.\new_image\frame62.png///Y3558.jpg
.\new_image\frame62.png///Y3559.jpg
.\new_image\frame62.png///Y3560.jpg
.\new_image\frame62.png///Y3561.jpg
.\new_image\frame62.png///Y3562.jpg
.\new_image\frame62.png///Y3563.jpg
.\new_image\frame62.png///Y3564.jpg
.\new_image\frame62.png///Y3

.\new_image\frame64.png///R3814.jpg
.\new_image\frame64.png///R3815.jpg
.\new_image\frame64.png///R3816.jpg
.\new_image\frame64.png///R3817.jpg
.\new_image\frame64.png///R3818.jpg
.\new_image\frame64.png///R3819.jpg
.\new_image\frame64.png///R3820.jpg
.\new_image\frame64.png///R3821.jpg
.\new_image\frame64.png///R3822.jpg
.\new_image\frame64.png///R3823.jpg
.\new_image\frame64.png///R3824.jpg
.\new_image\frame64.png///R3825.jpg
.\new_image\frame64.png///R3826.jpg
.\new_image\frame65.png///G3827.jpg
.\new_image\frame65.png///G3828.jpg
.\new_image\frame65.png///G3829.jpg
.\new_image\frame65.png///G3830.jpg
.\new_image\frame65.png///G3831.jpg
.\new_image\frame65.png///G3832.jpg
.\new_image\frame65.png///G3833.jpg
.\new_image\frame65.png///G3834.jpg
.\new_image\frame65.png///G3835.jpg
.\new_image\frame65.png///G3836.jpg
.\new_image\frame65.png///G3837.jpg
.\new_image\frame65.png///G3838.jpg
.\new_image\frame65.png///Y3839.jpg
.\new_image\frame65.png///Y3840.jpg
.\new_image\frame65.png///Y3

In [5]:
import os
import cv2
import numpy as np
import random
from PIL import Image

from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf

### 신호등 인식 모델

In [6]:
model_detection = tf.keras.models.load_model('traffic_light_detection_v16.h5')

#### 이미지 resize 해서 해당 폴더에 다시 저장

In [7]:
path_detection = 'vi_data2/' # 색 영역으로 검출한 모든 신호등 후보 영역 이미지

file_list_d = os.listdir(path_detection)

if ".DS_Store" in file_list_d:
    file_list_d.remove(".DS_Store")

data_detection = []

for A in file_list_d:
    if A == ".DS_Store":
        continue
        
    img_d = cv2.imread(path_detection + A)
    res_d = cv2.resize(img_d, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(path_detection + A, res_d)
    
for A in file_list_d:
    if A == ".DS_Store":
        continue
        
    img3 = Image.open(path_detection + A)
    data_detection.append(np.array(img3))
    img3.close()

In [8]:
data_detection = np.asarray(data_detection) / 255.0

In [9]:
prediction = model_detection.predict(data_detection)

In [10]:
light_order = {0 : "negative", 1 : "positive"}

light_detect_img = [] # 파일명, 예측 결과(negative, positive)
data_detection2 = [] # 신호등 이미지

for i, x in enumerate(prediction):
    pred_detect = ""
    
    if x >= 0.85:
        pred_detect = light_order[1]
    else:
        pred_detect = light_order[0]
    
    # 신호등으로 85퍼센트 이상 인식된 이미지만 추출
    if pred_detect == "positive":
        light_detect_img.append([file_list_d[i], light_order[1]])

        imgPath = path_detection + file_list_d[i]

        img_detection = Image.open(imgPath)
        data_detection2.append(np.array(img_detection)) # 신호등으로 인식된 이미지

In [11]:
len(data_detection2)

145

In [12]:
light_detect_img

[['frame0_1.jpg', 'positive'],
 ['frame0_2.jpg', 'positive'],
 ['frame0_3.jpg', 'positive'],
 ['frame0_4.jpg', 'positive'],
 ['frame0_5.jpg', 'positive'],
 ['frame0_6.jpg', 'positive'],
 ['frame11_64.jpg', 'positive'],
 ['frame11_66.jpg', 'positive'],
 ['frame11_67.jpg', 'positive'],
 ['frame11_95.jpg', 'positive'],
 ['frame12_108.jpg', 'positive'],
 ['frame12_109.jpg', 'positive'],
 ['frame12_110.jpg', 'positive'],
 ['frame13_146.jpg', 'positive'],
 ['frame13_153.jpg', 'positive'],
 ['frame13_154.jpg', 'positive'],
 ['frame13_161.jpg', 'positive'],
 ['frame15_191.jpg', 'positive'],
 ['frame15_192.jpg', 'positive'],
 ['frame16_216.jpg', 'positive'],
 ['frame17_252.jpg', 'positive'],
 ['frame1_22.jpg', 'positive'],
 ['frame1_23.jpg', 'positive'],
 ['frame1_24.jpg', 'positive'],
 ['frame1_25.jpg', 'positive'],
 ['frame2_267.jpg', 'positive'],
 ['frame2_268.jpg', 'positive'],
 ['frame37_855.jpg', 'positive'],
 ['frame37_896.jpg', 'positive'],
 ['frame37_897.jpg', 'positive'],
 ['frame37_8

### 신호등 색 분류

In [13]:
model = tf.keras.models.load_model('traffic_light_0315_2.h5')

In [14]:
# # 검출된 신호등 이미지를 입력 값으로 만들기
# Path2 = 'vi_data2/'
# file_list = os.listdir(Path2)
# file_list.sort()
# if ".DS_Store" in file_list:
#     file_list.remove(".DS_Store")

# data2 = []

# for A in file_list:
#     if A == ".DS_Store":
#         continue
#     try:
#         img2 = cv2.imread(Path2+A)
#         res2 = cv2.resize(img2, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
#         cv2.imwrite(Path2+A,res2)
#     except Exception as e:
#         print(str(e))
    
# for A in file_list:
#     if A == ".DS_Store":
#         continue
#     try:
#         img3 = Image.open(Path2 + A)
#         data2.append(np.array(img3))
#         img3.close()
#     except Exception as e:
#         print(str(e))

In [15]:
# data2

In [16]:
# data2 = np.asarray(data2) / 255.0
data_detection2 = np.asarray(data_detection2) / 255.0
prediction = model.predict(data_detection2)

In [17]:
prediction

array([[9.99915242e-01, 4.42958772e-05, 4.04123602e-05],
       [9.99971986e-01, 1.58490056e-05, 1.21340199e-05],
       [9.99949694e-01, 2.76767842e-05, 2.26821521e-05],
       [9.99936223e-01, 3.61119928e-05, 2.77081799e-05],
       [9.99986053e-01, 7.86714099e-06, 6.13206839e-06],
       [9.99842048e-01, 8.93347751e-05, 6.85393898e-05],
       [9.99915600e-01, 4.44717225e-05, 3.98821794e-05],
       [9.99931097e-01, 3.82876315e-05, 3.06245820e-05],
       [9.99931335e-01, 3.60753656e-05, 3.25931505e-05],
       [5.66749368e-03, 9.92033064e-01, 2.29939353e-03],
       [1.14216811e-04, 5.88093098e-05, 9.99826968e-01],
       [4.06109984e-06, 1.69750956e-06, 9.99994278e-01],
       [3.05435333e-06, 1.25684153e-06, 9.99995708e-01],
       [5.34500323e-05, 3.04322930e-05, 9.99916077e-01],
       [6.73249360e-07, 2.12568992e-07, 9.99999166e-01],
       [6.91935554e-07, 2.08469828e-07, 9.99999046e-01],
       [7.18271258e-06, 9.99990940e-01, 1.96519932e-06],
       [1.64150551e-05, 8.96056

In [18]:
len(first_detect) # 처음 색영역으로 신호등 후보 영역 검출한 개수

3978

In [19]:
light_order = {0 : "green", 1 : "red", 2 : "yellow", 3 : "unknown"}

# 신호등 파일명과 색을 한 리스트에 담기 
detect_img = [] 

for i, x in enumerate(prediction):
    count = 0
    for y in x:
        if y > 0.90:    # 90퍼센트 이상이면 인식
            count += 1
            
    if count > 0:
        print(str(i) + "번째 신호등: " + light_detect_img[i][0] + " : " + light_order[np.argmax(x)])
        detect_img.append([light_detect_img[i][0], light_order[np.argmax(x)]])
    else:
        print(str(i)+"번째 신호등: " + light_detect_img[i][0] + " : " + light_order[3])

0번째 신호등: frame0_1.jpg : green
1번째 신호등: frame0_2.jpg : green
2번째 신호등: frame0_3.jpg : green
3번째 신호등: frame0_4.jpg : green
4번째 신호등: frame0_5.jpg : green
5번째 신호등: frame0_6.jpg : green
6번째 신호등: frame11_64.jpg : green
7번째 신호등: frame11_66.jpg : green
8번째 신호등: frame11_67.jpg : green
9번째 신호등: frame11_95.jpg : red
10번째 신호등: frame12_108.jpg : yellow
11번째 신호등: frame12_109.jpg : yellow
12번째 신호등: frame12_110.jpg : yellow
13번째 신호등: frame13_146.jpg : yellow
14번째 신호등: frame13_153.jpg : yellow
15번째 신호등: frame13_154.jpg : yellow
16번째 신호등: frame13_161.jpg : red
17번째 신호등: frame15_191.jpg : yellow
18번째 신호등: frame15_192.jpg : yellow
19번째 신호등: frame16_216.jpg : red
20번째 신호등: frame17_252.jpg : red
21번째 신호등: frame1_22.jpg : green
22번째 신호등: frame1_23.jpg : green
23번째 신호등: frame1_24.jpg : green
24번째 신호등: frame1_25.jpg : green
25번째 신호등: frame2_267.jpg : green
26번째 신호등: frame2_268.jpg : green
27번째 신호등: frame37_855.jpg : yellow
28번째 신호등: frame37_896.jpg : red
29번째 신호등: frame37_897.jpg : red
30번째 신호등: frame37_898.jpg

In [20]:
detect_img

[['frame0_1.jpg', 'green'],
 ['frame0_2.jpg', 'green'],
 ['frame0_3.jpg', 'green'],
 ['frame0_4.jpg', 'green'],
 ['frame0_5.jpg', 'green'],
 ['frame0_6.jpg', 'green'],
 ['frame11_64.jpg', 'green'],
 ['frame11_66.jpg', 'green'],
 ['frame11_67.jpg', 'green'],
 ['frame11_95.jpg', 'red'],
 ['frame12_108.jpg', 'yellow'],
 ['frame12_109.jpg', 'yellow'],
 ['frame12_110.jpg', 'yellow'],
 ['frame13_146.jpg', 'yellow'],
 ['frame13_153.jpg', 'yellow'],
 ['frame13_154.jpg', 'yellow'],
 ['frame13_161.jpg', 'red'],
 ['frame15_191.jpg', 'yellow'],
 ['frame15_192.jpg', 'yellow'],
 ['frame16_216.jpg', 'red'],
 ['frame17_252.jpg', 'red'],
 ['frame1_22.jpg', 'green'],
 ['frame1_23.jpg', 'green'],
 ['frame1_24.jpg', 'green'],
 ['frame1_25.jpg', 'green'],
 ['frame2_267.jpg', 'green'],
 ['frame2_268.jpg', 'green'],
 ['frame37_855.jpg', 'yellow'],
 ['frame37_896.jpg', 'red'],
 ['frame37_897.jpg', 'red'],
 ['frame37_898.jpg', 'red'],
 ['frame38_929.jpg', 'yellow'],
 ['frame38_930.jpg', 'yellow'],
 ['frame38_9

In [21]:
print(len(file_list_d),len(detect_img)) # 모델에 적용한 이미지, 이미지 검출 코드에서 나온거

3978 144


In [22]:
xy = []
for i in range(0, len(detect_img)):
    for j in range(0, len(first_detect)):

        if first_detect[j][1] == detect_img[i][0].split(".")[0]:
            xy.append([first_detect[j][1], first_detect[j][0], detect_img[i][1]])
    #print(detect_img[i][0].split(".")[0])

In [23]:
xy

[['frame0_1', [589, 607, 568, 616], 'green'],
 ['frame0_2', [586, 606, 687, 747], 'green'],
 ['frame0_3', [579, 605, 801, 873], 'green'],
 ['frame0_4', [571, 597, 970, 1042], 'green'],
 ['frame0_5', [32, 93, 658, 820], 'green'],
 ['frame0_6', [4, 61, 1265, 1421], 'green'],
 ['frame11_64', [620, 632, 675, 705], 'green'],
 ['frame11_66', [587, 601, 1046, 1088], 'green'],
 ['frame11_67', [580, 594, 1144, 1183], 'green'],
 ['frame11_95', [311, 325, 1649, 1685], 'red'],
 ['frame12_108', [514, 544, 926, 998], 'yellow'],
 ['frame12_109', [509, 529, 1074, 1135], 'yellow'],
 ['frame12_110', [500, 521, 1220, 1275], 'yellow'],
 ['frame13_146', [497, 521, 357, 415], 'yellow'],
 ['frame13_153', [277, 321, 918, 1031], 'yellow'],
 ['frame13_154', [263, 302, 1226, 1328], 'yellow'],
 ['frame13_161', [500, 520, 369, 419], 'red'],
 ['frame15_191', [607, 625, 1079, 1123], 'yellow'],
 ['frame15_192', [605, 619, 1190, 1237], 'yellow'],
 ['frame16_216', [497, 533, 1134, 1236], 'red'],
 ['frame17_252', [52, 1

In [24]:
array = []
for i in range(0, len(xy)):
    coor = xy[i][1]
    color = xy[i][2]
    for j in range(i+1, len(xy)):
        if xy[i][0] == xy[j][0]:
            coor = xy[j][1]
            color = xy[j][2]
    if [xy[i][0], coor, color] not in array:
        array.append([xy[i][0], coor, color])
    #print(xy[i])

In [25]:
array

[['frame0_1', [589, 607, 568, 616], 'green'],
 ['frame0_2', [586, 606, 687, 747], 'green'],
 ['frame0_3', [579, 605, 801, 873], 'green'],
 ['frame0_4', [571, 597, 970, 1042], 'green'],
 ['frame0_5', [32, 93, 658, 820], 'green'],
 ['frame0_6', [4, 61, 1265, 1421], 'green'],
 ['frame11_64', [620, 632, 675, 705], 'green'],
 ['frame11_66', [587, 601, 1046, 1088], 'green'],
 ['frame11_67', [580, 594, 1144, 1183], 'green'],
 ['frame11_95', [311, 325, 1649, 1685], 'red'],
 ['frame12_108', [514, 544, 926, 998], 'yellow'],
 ['frame12_109', [509, 529, 1074, 1135], 'yellow'],
 ['frame12_110', [500, 521, 1220, 1275], 'yellow'],
 ['frame13_146', [497, 521, 357, 415], 'yellow'],
 ['frame13_153', [277, 321, 918, 1031], 'yellow'],
 ['frame13_154', [263, 302, 1226, 1328], 'yellow'],
 ['frame13_161', [500, 520, 369, 419], 'red'],
 ['frame15_191', [607, 625, 1079, 1123], 'yellow'],
 ['frame15_192', [605, 619, 1190, 1237], 'yellow'],
 ['frame16_216', [497, 533, 1134, 1236], 'red'],
 ['frame17_252', [52, 1

# array에 담긴 파일명으로 원천 이미지에 시각화

- path 경로에서 원천 이미지 찾아서 그 이미지에다가 rectangle 그리는 거여서 덮어씌워짐  
- 덮어씌워지니까 원천 이미지는 꼭 백업해둘것

In [26]:
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 

for a,b,c in array:
    print(a,b,c)
    imgname = a.split('_')[0] + '.png'
    print(path,imgname)

frame0_1 [589, 607, 568, 616] green
./new_image/ frame0.png
frame0_2 [586, 606, 687, 747] green
./new_image/ frame0.png
frame0_3 [579, 605, 801, 873] green
./new_image/ frame0.png
frame0_4 [571, 597, 970, 1042] green
./new_image/ frame0.png
frame0_5 [32, 93, 658, 820] green
./new_image/ frame0.png
frame0_6 [4, 61, 1265, 1421] green
./new_image/ frame0.png
frame11_64 [620, 632, 675, 705] green
./new_image/ frame11.png
frame11_66 [587, 601, 1046, 1088] green
./new_image/ frame11.png
frame11_67 [580, 594, 1144, 1183] green
./new_image/ frame11.png
frame11_95 [311, 325, 1649, 1685] red
./new_image/ frame11.png
frame12_108 [514, 544, 926, 998] yellow
./new_image/ frame12.png
frame12_109 [509, 529, 1074, 1135] yellow
./new_image/ frame12.png
frame12_110 [500, 521, 1220, 1275] yellow
./new_image/ frame12.png
frame13_146 [497, 521, 357, 415] yellow
./new_image/ frame13.png
frame13_153 [277, 321, 918, 1031] yellow
./new_image/ frame13.png
frame13_154 [263, 302, 1226, 1328] yellow
./new_image/ f

In [27]:
# 원천이미지 있는 디렉토리 경로
# 해당 디렉토리 백업해두기
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 


for i in range(len(array)):
    a, b, c = array[i]
    
    frame = a.split('_')[0]
    imgname = frame + '.png'
    
    for j in range(i + 1, len(array)):
        a1, b1, c1 = array[j]
        a1 = a1.split('_')[0]
        
        if frame == a1: # 같은 이미지 내에서만 비교
            if c == 'red' or c == 'yellow':
                if c1 == 'red' or c1 == 'yellow':
                    if abs(b[2] - b1[2]) <= 15:
                        # 비슷한 영역이 두번 검출되었으므로 객체 하나 삭제(yellow 삭제)
                        if c == 'red' or c == c1:
                            array[j][1] = [0, 0, 0, 0]
                        else:
                            b = 0, 0, 0, 0
        else:
            break

            
    img = cv2.imread(path + imgname)
    
    y, h, x, w = b
    w = w - x
    h = h - y
    text = c 
    org = (x, y - 8)
    
    if x > 0 and y > 0 and w > 0 and h > 0:
        cv2.rectangle(img, (x, y, w, h), (139, 255, 224), 2)
        cv2.putText(img, text, org, font, 0.7, (139, 255, 224), 2)
        
    # 기존 원천 이미지에 덮어쓰기 되는거니까 주의!!!
    cv2.imwrite(path + imgname, img)
    

# 원천 이미지 연속 출력 (검출된 내용 반영)

#### 숫자 정렬용 라이브러리 설치

In [28]:
# !pip install natsort

In [29]:
import sys
import glob
import cv2

import natsort # 숫자 정렬용 라이브러리

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
img_files = glob.glob('.\\new_image\\*.png')
# img_files.sort()

# 문자열로 인식해서 [1, 10, ..., 2] 이런식으로 나오는 오류
# natsort 사용하여 [1, 2, ..., 10] 순으로 나올 수 있도록 정렬
img_files = natsort.natsorted(img_files)

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
	# ESC가 입력되면 break
    cv2.imshow('video_test', img)
    if cv2.waitKey(1000) == 27:     
        break

	# index가 이미지 리스트보다 커지거나 같아지면 다시 0으로
    index += 1      
    
    #if index >= count :
        #index = 0 

cv2.waitKey(0)    
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# 동영상으로 저장

In [30]:
import numpy as np
import glob
 
img_array = []

filename_list = glob.glob('.\\new_image\\*.png')
filename_list = natsort.natsorted(filename_list)

for filename in filename_list:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter('video_test.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()